# EM アルゴリズムによる GMM サンプル

**データサンプル**

一番結果が出やすそうな [3ヶ国の気候データ](https://github.com/KameKingdom/-------11-/blob/main/3_countries_climate.csv)を使用する
カラム１：平均気温，　カラム２：降水量

結果悲惨な分布だったのでプログラムで作成したデータに変更

またプログラムを書き換えて

In [28]:
import tensorflow as tf
import numpy as np
import csv
import os
from scipy.special import cbrt
import scipy as sp
import subprocess
import matplotlib.pyplot as pl
from pandas import DataFrame, Series

directory = "C:/Users/yudai/Desktop/KGU/3年生春学期/音楽情報処理/音楽情報処理第11回/"  # データディレクトリ
input_filename = "cluster_data.csv" #データ名

def get_faithful_data():
    f = open(os.path.join(directory, input_filename), "r")
    reader = csv.reader(f)
    header = next(reader)
    ret = []
    for row in reader:
         x = float(row[0])
         y = float(row[1])
         ret.append([x, y])
    return np.array(ret, dtype = np.float64)

faithful = get_faithful_data()

def draw_ring(mu, dev):
    angles = np.linspace(0, 2 * np.pi, 100)
    x = mu[0] + dev[0] * np.cos(angles)
    y = mu[1] + dev[1] * np.sin(angles)
    pl.plot(x, y, 'r-')

def draw(mu, prec):
    pl.clf()
    pl.plot(faithful[:, 0], faithful[:, 1], 'b+')
    pl.plot(mu[:, 0], mu[:, 1], 'go')
    for i in range(len(mu)):
        draw_ring(mu[i], np.sqrt(1. / prec[i]))
    pl.xlim(0, 10)
    pl.ylim(0, 10)
    pl.pause(.1)

def calc_estep(pi, mu, prec):
    lpi = np.log(pi)
    lprec = np.log(prec)
    lz = lpi[None, :] + .5 * lprec.sum(1)[None, :] - .5 * (prec[None, :, :] *
         (faithful[:, None, :] - mu[None, :, :]) ** 2).sum(2)
    lz -= sp.special.logsumexp(lz, axis=1)[:, None]
    return np.exp(lz)

def calc_mstep(z):
    pi = z.sum(0) / z.sum()
    mu = (z[:, :, None] * faithful[:, None, :]).sum(0) / z.sum(0)[:, None]
    prec = z.sum(0)[:, None] / (z[:, :, None] * ((faithful[:, None, :] -
           mu[None, :, :]) ** 2)).sum(0)
    return pi, mu, prec

def evaluate_metrics(history, mu):
    convergence_speed = np.mean(np.abs(np.diff(history)))
    separation_degree = np.mean(np.linalg.norm(np.diff(mu, axis=0), axis=1))
    robustness_to_outliers = np.mean(np.std([np.linalg.norm(faithful - mu_i, axis=1) for mu_i in mu], axis=1))

    print("Convergence speed: ", convergence_speed)
    print("Separation degree: ", separation_degree)
    print("Robustness to outliers: ", robustness_to_outliers)

def process_em(z, nstep):
    history = []
    def kl_loss(): 
        return np.abs(np.sum(pi) - 1) # 計算方法

    for i in range(nstep):
        pi, mu, prec = calc_mstep(z)
        z = calc_estep(pi, mu, prec)
        kl_loss_value = kl_loss() # Calculate KL divergence
        history.append(kl_loss_value) # Append the KL divergence value to history
        draw(mu, prec)
    evaluate_metrics(history, mu)


def process_sgd(z, nstep, use_adam=True):
    history = [] # Initialize history
    pi, mu, prec = calc_mstep(z)
    lpi = tf.Variable(np.log(pi))
    mu = tf.Variable(mu)
    lprec = tf.Variable(np.log(prec))
    faith = tf.constant(faithful)
    if use_adam:
        opt = tf.keras.optimizers.Adam(learning_rate=.1)
    else:
        opt = tf.keras.optimizers.RMSprop(learning_rate=.03)
    for i in range(nstep):
        def kl_loss():
            penalty = tf.abs((tf.reduce_sum(tf.exp(lpi)) - 1))
            lpi_normal = lpi - tf.reduce_logsumexp(lpi)
            likelihood = (lpi_normal + .5 * tf.reduce_sum(lprec, axis=1))[None, :]
            likelihood -= .5 * tf.reduce_sum(tf.exp(lprec)[None, :, :] *
                          ((faith[:, None, :] - mu[None, :, :]) ** 2), axis=2)
            return penalty - tf.reduce_sum(tf.reduce_logsumexp(likelihood, axis=1))
        opt.minimize(kl_loss, var_list=[lpi, mu, lprec])
        kl_loss_value = kl_loss() # Calculate KL divergence
        history.append(kl_loss_value) # Append the KL divergence value to history
        draw(mu.numpy(), tf.exp(lprec).numpy())
    evaluate_metrics(history, mu.numpy())


nsample = len(faithful)
nclass = 3
nstep = 400
z = np.random.dirichlet(np.ones(nclass), nsample)
use_em = False
use_adam = True
if use_em:
    process_em(z, nstep)
else:
    process_sgd(z, nstep, use_adam)


Convergence speed:  2.2209593971811916
Separation degree:  5.81859307638306
Robustness to outliers:  2.625567347837022
